In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
# Configs
%config InlineBackend.figure_format ='retina'
%matplotlib inline

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn import base, neural_network, neighbors, feature_extraction, datasets, linear_model, metrics, model_selection, preprocessing, svm, ensemble, decomposition, feature_extraction, utils

import keras


Using TensorFlow backend.


In [10]:
# Load Bridge helper classes
%run helpers/bridge.ipynb
%run helpers/load-data.ipynb
%run helpers/train-eval-utils.ipynb

In [13]:
# Loads & precomputes all the data, takes about 2.5min
data = load_deals()

In [26]:
# Concatenating features
X = [np.array([h.feature_array() for h in d.hands]).flatten()
     for d in data]

# Winners in a particular strain played from N
y = [d.wnrs[strains.index('NT')]['N'] for d in data]
y_suit = [d.wnrs[strains.index('s')]['N'] for d in data]

In [ ]:
# PREVIOUS PERFORMANCE with old features, smaller dataset

# Features: Globally-defined X
#     Targets: Globally-defined y


#         M: Skl 1layNN #  test MAE: 0.76 - tr MAE: 0.72 - T: 256.4s # 40.2|86.2|97.8
#         M: HistGB     #  test MAE: 0.84 - tr MAE: 0.82 - T:   6.6s # 36.3|83.6|97.0
#         M: LinearRegr #  test MAE: 0.97 - tr MAE: 0.96 - T:   0.7s # 32.3|77.6|94.8


In [ ]:
#   10 iter = 1.5 min
#        M: Skl 1layNN #  test MAE: 0.80 - tr MAE: 0.79 - T: 59.6s # 38.0|84.9|97.5

#    200 iterations = 45 mins
#        M: Skl 1layNN #  test MAE: 0.73 - tr MAE: 0.70 - T: 2753.0s # 41.2|87.9|98.4

# 10 iter = 6 min
#        M: 4 layer 256 #  test MAE: 0.70 - tr MAE: 0.67 - T: 351.3s # 43.0|88.9|98.4
#        M: 7 layer 256 #  test MAE: 0.68 - tr MAE: 0.65 - T: 445.1s # 44.2|89.4|98.5
#           12 layer was same as 7, but worse training time

## results with mask instead
#        M: 7 layer 256 #  test MAE: 0.77 - tr MAE: 0.67 - T: 598.2s # 39.5|85.9|97.6


#    convergence = 44 iterations
#        M: Sklearn-NN #  test MAE: 0.61 - tr MAE: 0.50 - T: 6613.4s # 49.0|91.8|98.8


In [31]:
m = train_eval_dds({'m': neural_network.MLPRegressor(hidden_layer_sizes=(256,) * 7,
                    max_iter=300, early_stopping=True, warm_start=True), 
                    'desc': 'Sklearn-NN'}, ret_model=True) # 4 min

Features: Globally-defined X
    Targets: Globally-defined y

        M: Sklearn-NN #  test MAE: 0.61 - tr MAE: 0.50 - T: 6613.4s # 49.0|91.8|98.8


In [35]:
m[0].n_iter_

44

In [32]:
from sklearn.externals import joblib
joblib.dump(m, 'models/ftarrays_to_dds_NT_sklearn.pkl') 

['models/ftarrays_to_dds_NT_sklearn.pkl']

In [33]:
m2 = train_eval_dds({'m': neural_network.MLPRegressor(hidden_layer_sizes=(256,) * 7,
                    max_iter=100, early_stopping=True, warm_start=True), 
                    'desc': 'Sklearn-NN'}, targets = [(np.array(y_suit), 'SPADE WINNERS')], ret_model=True) # 4 min

SyntaxError: invalid syntax (<ipython-input-33-39bad15f50db>, line 3)

In [ ]:
joblib.dump(m2, 'models/ftarrays_to_dds_s_sklearn.pkl') 

In [27]:
train_eval_dds({'m': seq_model, 'desc': 'MLPNadamTan',
            'b_args': {'activation': 'tanh', 'optimizer': keras.optimizers.Nadam(),
                       'layers_no': 6, },
            'f_args': {'batch_size': 32, 'epochs': 5, 'verbose': 1}})  # 3min on CPU

Instructions for updating:
Colocations handled automatically by placer.
Features: Globally-defined X
    Targets: Globally-defined y

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
337278/337278 [==============================] - 228s 677us/step - loss: 1.7861 - mean_absolute_error: 1.0485
Epoch 2/5
337278/337278 [==============================] - 224s 663us/step - loss: 1.6917 - mean_absolute_error: 1.0204
Epoch 3/5
337278/337278 [==============================] - 212s 628us/step - loss: 1.6718 - mean_absolute_error: 1.0155
Epoch 4/5
182016/337278 [===============>..............] - ETA: 1:42 - loss: 1.6643 - mean_absolute_error: 1.0130

KeyboardInterrupt: 